In [1]:

import torch


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/Caskroom/miniconda/base/envs/ML-Practice/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/Caskroom/miniconda/base/envs/ML-Practice/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/local/Caskroom/miniconda/base/envs/ML-Practice/lib/python3.11/site-packages/i

In [6]:
t = torch.Tensor([1,2,2,3])
t

tensor([1., 2., 2., 3.])

In [ ]:
nnc = None 

membank = None

In [12]:
n_loc: int = 10  # N
dim_loc: int = 20  # M 

memmat = torch.rand(size=(n_loc, dim_loc))
memmat.size()

torch.Size([10, 20])

In [13]:
# read_head -> w_t vec weights with N entries 
# read_vector r_t = convex combination of mem elements with weights = w_t

In [37]:
w = torch.arange(0.1,.6,0.1)
w = w[:, None]
print(w.size())
memmat = torch.randint(10, (5,2))
print(memmat)
print(w)
print(w * memmat)
torch.sum(w * memmat, dim=0)




torch.Size([5, 1])
tensor([[0, 0],
        [6, 8],
        [6, 5],
        [9, 2],
        [7, 5]])
tensor([[0.1000],
        [0.2000],
        [0.3000],
        [0.4000],
        [0.5000]])
tensor([[0.0000, 0.0000],
        [1.2000, 1.6000],
        [1.8000, 1.5000],
        [3.6000, 0.8000],
        [3.5000, 2.5000]])


tensor([10.1000,  6.4000])

In [38]:
class ReadHead(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def forward(self, mem_mat: torch.Tensor):
        """Emit read vector r_t"""
        
        # product weights w_t
        # emit reaad vector r_t
        weights = torch.rand(mem_mat.size()[0])
        weights = weights[:, None]
        return torch.sum(weights * mem_mat, dim=0)

In [39]:
rh = ReadHead()

In [41]:
memmat

tensor([[0, 0],
        [6, 8],
        [6, 5],
        [9, 2],
        [7, 5]])

In [44]:

rh.forward(memmat).gradient()

AttributeError: 'Tensor' object has no attribute 'gradient'

In [ ]:
class WriteHead(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def forward(self, memmat: torch.Tensor):
        """Write head adjusts memory matrix."""

        weights = torch.rand(memmat.size()[0])
        v_erase =  torch.rand(memmat.size()[1])
        v_add =  torch.rand(memmat.size()[1])
        weights = weights[:, None]
        ones = torch.ones(memmat.size()[1])
        for i in range(len(memmat.size()[0])):
            memmat[i, :] = memmat[i, :] * (ones - weights[i] * v_erase)
            memmat[i, :] = memmat[i, :] * (ones - weights[i] * v_add)

        return None




In [52]:
e_mat = e.repeat(4,1)
e_mat


tensor([[0.2607, 0.6224, 0.4122],
        [0.2607, 0.6224, 0.4122],
        [0.2607, 0.6224, 0.4122],
        [0.2607, 0.6224, 0.4122]])

In [ ]:
# free params controller
SIZE_OF_MEM = 10
NUM_READ_HEADS = 5
NUM_WRITE_HEADS = 5
LOC_SHIFT_RANGE = list(range(1,6))


In [90]:
from typing import Optional
class MemoryBank(torch.nn.Module):
    def __init__(self, num_vectors: int, vec_dim: int):
        super(MemoryBank, self).__init__()
        self.num_vec = num_vectors
        self.vec_dim = vec_dim
        self.data: torch.Tensor | None = None

    def init_state(self, batch_size, device):
        self.data = torch.zeros(batch_size, self.num_vec, self.vec_dim).to(device)


In [91]:
class NeuralNetController(torch.nn.Module):
    """Some kind of Recurrent net or feedforward net."""
    # typically LSTM
    def __init__(self, h_size=20, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_size = h_size
        self.lstm_cell = torch.nn.LSTM(input_size=1, hidden_size=h_size, batch_first=True)
        self.memory_bank: MemoryBank = MemoryBank(10, h_size)

    
    def forward(self):
        # outputs k_t, beta_t, g_t, s_t, \gamma_t
        return


In [54]:
# dataset
copy_dataset = None
sort_dataset = None

In [73]:
# copy dataset 
copy_vecs = torch.randint(2, (1000, 8))
copy_vecs[1, :]


tensor([1, 1, 1, 0, 1, 1, 0, 0])

In [77]:
from torch.utils.data import Dataset
class CopyDataset(Dataset):
    def __init__(self, len, delim: int=-1):
        super().__init__()
        self.delim = delim
        self.data = torch.randint(2, (len, 8))
        self.data = torch.column_stack([self.data, torch.ones(self.data.size()[0]) * self.delim])
    
    def __getitem__(self, idx: int):
        return (self.data[idx, :], self.data[idx, :-1])

In [89]:
cop_data = CopyDataset(10)
cop_data[8]

(tensor([ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  0., -1.]),
 tensor([0., 0., 0., 0., 1., 1., 1., 0.]))

- for each head, be it a read or write head, the addressing mechanism is implemented
=> for each head, the controller needs to produce
  + key vector $k_t$
  + key strength $\beta_t$
  + interpolation gate $g_t$
  + shift weighting $s_t$
  + sharpening factor $\gamma_t$

In [99]:
x = torch.randint(3, (5,3))
y = torch.randint(3, (3,))
x, y, torch.cat([x] + y, dim=1)

TypeError: can only concatenate list (not "Tensor") to list